Data Preprocessing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data=pd.read_csv('my_data.csv')

In [3]:
print(data.head())
print(data.shape)

        name    city  gender profession   age  cgpa  placed
0  moriarity     NaN  female        phd  28.0  5.94       1
1  moriarity  asgard     NaN   bachelor  50.0  8.55       0
2     holmes     NaN  female    masters  18.0  5.56       0
3        sam     NaN    male   bachelor  25.0  8.57       1
4        sam     NaN    male   bachelor  19.0  8.76       1
(1100, 7)


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        1009 non-null   object 
 1   city        905 non-null    object 
 2   gender      994 non-null    object 
 3   profession  921 non-null    object 
 4   age         982 non-null    float64
 5   cgpa        962 non-null    float64
 6   placed      1100 non-null   int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 60.3+ KB


Creating a independent(y) and dependent variable(X)
dependent variable are those in which outcomes depended
indenpendent variables are those on which outcomes are not depende(outcome itself)

In [5]:
#iloc index location , ['starting row':'ending row','starting column':'ending column']
X=data.iloc[:,:-1].values

In [6]:
X

array([['moriarity', nan, 'female', 'phd', 28.0, 5.94],
       ['moriarity', 'asgard', nan, 'bachelor', 50.0, 8.55],
       ['holmes', nan, 'female', 'masters', 18.0, 5.56],
       ...,
       ['sam', 'asgard', 'male', 'bachelor', 30.0, 7.88],
       ['dean', 'gotham', 'male', 'masters', 28.0, nan],
       ['sam', 'asgard', nan, 'masters', 24.0, 8.94]], dtype=object)

In [7]:
y=data.iloc[:,-1].values

In [8]:
y

array([1, 0, 0, ..., 0, 1, 1], dtype=int64)

In [9]:
#how to handle missing data by filling with mean of that column that has numerical data
from sklearn.impute import SimpleImputer
SimpleImputer=SimpleImputer(missing_values=np.nan,strategy='mean')
SimpleImputer=SimpleImputer.fit(X[:,4:6]) #fit only on columns which have missing data
X[:,4:6]=SimpleImputer.transform(X[:,4:6])

In [10]:
print(X)

#How to handle non numerical data(categorical data) missi ng data by filling with most frequent value of that column
from sklearn.impute import SimpleImputer
SimpleImputer=SimpleImputer(missing_values=np.nan,strategy='most_frequent')
SimpleImputer=SimpleImputer.fit(X[:,1:4]) #fit only on columns which have missing data
X[:,1:4]=SimpleImputer.transform(X[:,1:4])

print('---***---'*6)
print('---***---'*6)
print('---***---'*6)

print(X)

[['moriarity' nan 'female' 'phd' 28.0 5.94]
 ['moriarity' 'asgard' nan 'bachelor' 50.0 8.55]
 ['holmes' nan 'female' 'masters' 18.0 5.56]
 ...
 ['sam' 'asgard' 'male' 'bachelor' 30.0 7.88]
 ['dean' 'gotham' 'male' 'masters' 28.0 7.609033264033264]
 ['sam' 'asgard' nan 'masters' 24.0 8.94]]
---***------***------***------***------***------***---
---***------***------***------***------***------***---
---***------***------***------***------***------***---
[['moriarity' 'wakanda' 'female' 'phd' 28.0 5.94]
 ['moriarity' 'asgard' 'male' 'bachelor' 50.0 8.55]
 ['holmes' 'wakanda' 'female' 'masters' 18.0 5.56]
 ...
 ['sam' 'asgard' 'male' 'bachelor' 30.0 7.88]
 ['dean' 'gotham' 'male' 'masters' 28.0 7.609033264033264]
 ['sam' 'asgard' 'male' 'masters' 24.0 8.94]]


In [11]:
#to delete rows with missing data
#data.dropna(inplace=True)
#to delete columns with missing data
#data.dropna(axis=1,inplace=True)

In [12]:
data['city'].value_counts()

city
wakanda       391
gotham        243
asgard        152
purgatory     119
Name: count, dtype: int64

Now suppose the city column is important for our model training, we will have to convert the string data to interger / numbeeric type for machine to understand.

In [13]:
from sklearn.preprocessing import LabelEncoder
labelencoder_X=LabelEncoder()
X[:,1]=labelencoder_X.fit_transform(X[:,1])
print(X)
print("Number of different type of city in data",data['city'].nunique())

[['moriarity' 3 'female' 'phd' 28.0 5.94]
 ['moriarity' 0 'male' 'bachelor' 50.0 8.55]
 ['holmes' 3 'female' 'masters' 18.0 5.56]
 ...
 ['sam' 0 'male' 'bachelor' 30.0 7.88]
 ['dean' 1 'male' 'masters' 28.0 7.609033264033264]
 ['sam' 0 'male' 'masters' 24.0 8.94]]
Number of different type of city in data 4


Now if we see the city column has been converted in number type 0,1,2,3 but this could be misunderstood by machine considereing 1 as higher presidence then 0 so we will create dummy metrix.

In machine learning, a "dummy matrix" usually refers to a matrix of dummy variables, which are binary (0/1) indicators used to represent categorical data numerically.

data   | wakanda |  gotham |  asgard |  purgatory

Wakanda   |  1    |    0   |     0    |    0
gotham    |  0    |    1   |     0    |    0

In [14]:
from sklearn.preprocessing import OneHotEncoder

encoder=OneHotEncoder(sparse=False)  # sparse=False returns a dense array

# Apply encoder only to the categorical column (column index 1)

X_encoded=encoder.fit_transform((X[:,[1]]))

# Combine encoded column with the rest of the data

X_final=np.concatenate((X[:,0:1],X_encoded,X[:,2:]),axis=1)

X=X_final
np.set_printoptions(suppress=True)
print(X)

"""OneHotEncoder=OneHotEncoder(categorical_features=[1])
X=OneHotEncoder.fit_transform(X).toarray()
np.set_printoptions(suppress=True)"""
print(X)

[['moriarity' 0.0 0.0 ... 'phd' 28.0 5.94]
 ['moriarity' 1.0 0.0 ... 'bachelor' 50.0 8.55]
 ['holmes' 0.0 0.0 ... 'masters' 18.0 5.56]
 ...
 ['sam' 1.0 0.0 ... 'bachelor' 30.0 7.88]
 ['dean' 0.0 1.0 ... 'masters' 28.0 7.609033264033264]
 ['sam' 1.0 0.0 ... 'masters' 24.0 8.94]]
[['moriarity' 0.0 0.0 ... 'phd' 28.0 5.94]
 ['moriarity' 1.0 0.0 ... 'bachelor' 50.0 8.55]
 ['holmes' 0.0 0.0 ... 'masters' 18.0 5.56]
 ...
 ['sam' 1.0 0.0 ... 'bachelor' 30.0 7.88]
 ['dean' 0.0 1.0 ... 'masters' 28.0 7.609033264033264]
 ['sam' 1.0 0.0 ... 'masters' 24.0 8.94]]


c:\Users\amarj\Python_env_Scripts\Deep_learning_env\tf_env\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
#splitting the dataset into training set and test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0) 
print(X_train)
print(X_test)
print(y_train)      
print(y_test)

[['watson' 1.0 0.0 ... 'masters' 27.0 7.79]
 ['sherlock' 0.0 0.0 ... 'bachelor' 49.0 9.38]
 ['sam' 1.0 0.0 ... 'bachelor' 19.0 7.44]
 ...
 ['dean' 0.0 1.0 ... 'masters' 22.0 9.22]
 ['sam' 0.0 0.0 ... 'masters' 29.0 8.48]
 ['sam' 0.0 0.0 ... 'masters' 24.0 9.21]]
[['sam' 0.0 0.0 ... 'bachelor' 27.0 8.36]
 ['sam' 1.0 0.0 ... 'bachelor' 22.0 9.4]
 ['sam' 0.0 0.0 ... 'bachelor' 45.0 7.17]
 ...
 ['sam' 0.0 1.0 ... 'bachelor' 28.429735234215887 9.27]
 ['moriarity' 0.0 0.0 ... 'bachelor' 23.0 8.26]
 ['sam' 0.0 0.0 ... 'bachelor' 21.0 4.59]]
[1 0 1 1 1 1 1 1 1 1 0 0 1 0 1 1 0 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0 1 1 0 0
 0 1 1 1 0 1 0 1 1 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 1 1 1 0 0 0 0 1 0 0 1 1 0
 0 1 0 1 1 0 1 1 0 0 1 0 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 0 0 1 0 0 0 0
 1 0 0 0 0 1 1 0 1 0 0 1 1 1 1 0 1 0 1 0 1 1 1 1 0 1 0 0 0 1 0 0 1 1 1 1 1
 0 1 0 1 1 0 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1
 1 1 1 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 0 0
 1 0 1 1 1

In [18]:
#Feature Scaling i.e standard deviation and normalization i.e handling varying data(big and small ) numbers
from sklearn.preprocessing import StandardScaler
Scale=StandardScaler()
X_train=Scale.fit_transform(X_train)
X_test=Scale.transform(X_test)
print(X_train)
print(X_test)

ValueError: could not convert string to float: 'watson'